# Dependencias

In [1]:
import numpy as np
import os
import tensorflow as tf
import inkml2img
import re
import json
import xml.etree.ElementTree as ET
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from jiwer import wer

# Preprocesamiento

In [ ]:
# En esta celda se convierten los archivos inkml a .jpg con ayuda del modulo inkml2img
# PATH indica la ubicacion de los archivos .inkml
PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\TrainINKML"
# DIR_PATH indica el directorio donde se guardarán las imagenes
DIR_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\images"
# se obtiene el nombre de las carpetas que contienen los archivos .inkml
dataSets = os.listdir(PATH)
# se genera una lista con listas vacias (la cantidad de carpetas en dataSets)
arr = [[] for _ in range(len(dataSets))]
print(dataSets)
# se itera por cada carpeta de archivos .inkml
for i in range(len(dataSets)):
    # se agrega a la lista los archivos en determinada carpeta (la de la iteracion actual)
    arr[i] = os.listdir(PATH+'\\'+dataSets[i])
    # se itera por los archivos .inkml de la carpeta
    for t in arr[i]:
    # si la extension no es .lg
    if ('.lg' not in t):
        # se llama a la funcion inkml2img del modulo hominimo, especificando la ruta del inkml y la ruta a guardar la imagen
        inkml2img.inkml2img(PATH+'\\'+dataSets[i]+'\\'+t,DIR_PATH+'\\'+t[:-5]+'jpg')
        #print(t)

['expressmatch', 'extension', 'HAMEX', 'KAIST', 'MathBrush', 'MfrDB']


"    for t in arr[i]:\n    # si la extension no es .lg\n    if ('.lg' not in t):\n        # se llama a la funcion inkml2img del modulo hominimo, especificando la ruta del inkml y la ruta a guardar la imagen\n        inkml2img.inkml2img(PATH+'\\'+dataSets[i]+'\\'+t,DIR_PATH+'\\'+t[:-5]+'jpg')\n        print(t)"

In [ ]:
# en esta celda se obtienen los labels (el ground truth en latex) asociado a cada archivo .inkml
# Paths indicando la direccion donde guardar y leer los archivos del conjunto de datos
DIR_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\TrainINKML"
LABELS_PATH = r"/content/drive/My Drive/Colab Notebooks/TrainINKML/"
# diccionario donde el ground truth se guardara para cada archivo
items = {}
index = 0
count = 0
# se itera por la cantidad de carpetas que contienen archivos .inkml
for t in arr:
    # se itera por cada archivo inkml
    for i in t:
        if (".lg" not in i):
            # se obtiene el path del archivo concatenando DIR_PATH con el nombre de la carpeta y del archivo
            imgPath = DIR_PATH+"\\"+ dataSets[index]+ "\\"+i
            # dado que .inkml es un tipo de XML, tiene una estructura de arbol, por lo que se utiliza 
            # la funcion ET.parse para hacer uso de esta estructura
            tree = ET.parse(imgPath)
            # se otbiene la raiz del archivo (primera etiqueta en el .inkml)
            root = tree.getroot()
            count =0
            # se itera por cada hijo de la raiz
            for item in root:
                # si el hijo posee texto entonces es posible que contenga el ground truth (label) deseado
                if item.text:
                    # existen tres posibilidades debido a la forma en que fueron codificados los archivos en distintas carpetas
                    # el primero es que el groud truth en latex se encuetre entre simbolos de dolar $
                    if "$" in item.text:
                        current = item.text
                        # se añade como key del diccionario el nombre del archivo y se le asocia el ground truth en latex
                        items[DIR_PATH+"\\"+ dataSets[index]+ "\\"+i] = current[1:-1]
                        break
                    # para la carpeta MathBrush cuyo formato de groundTruth es distinto
                    if "\\" in item.text and dataSets[index] == "MathBrush":
                        current = item.text
                        items[DIR_PATH+"\\"+ dataSets[index]+ "\\"+i] = current[1:-1]
                        break
                    # para la carpeta KAIST el latex se encuentra sin elementos externos, por lo que se agrega sin mas
                    if (count ==1 and dataSets[index]=="KAIST" and len(item.text)>2):
                        current = item.text
                        items[DIR_PATH+"\\"+ dataSets[index]+ "\\"+i] = current
                        break
                count +=1
    index +=1 
# se verifica que el comando log no se encuentre como una secuencia de letras separadas
for key in items:
    if ('l o g' in items[key]):
        # si se encuentra se reemplaza por su respectivo comando en latex correcto
        items[key] = items[key].replace('l o g', '\\log')
# se guarda el diccionario en formato json
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'w') as f:
    json.dump(items, f, indent=4)

In [ ]:
# se realiza un filtrado de los groud truth para separar cada comando, letra y numero presente en el dataset
# establece los comandos a borrar, ya que no proporcionan informacion relevante para la ecuacion
delete = ['\\Bigg','\\left','\\right','\\Big','\\mathrm']
# comandos a reemplazar por el comando latex correcto
replace = {'\\to':'\\rightarrow', '\\gt':'>', '\\lt':'<'}
# simbolos a los que se les añade espacios en blanco antes y despues para un mejor tratamiento
add = ['_','{','}','=','(',')','-','+','^','[',']', ',']
count = 0
# se iteran los archivos .inkml, items representa el mismo diccionario guardado en labels.json
for key in items:
    # si existe alguno de los comandos descritos en las listas y diccionario anteriores se realizan la soperaciones
    # correspondientes
    for dele in delete:
        if dele in items[key]:
            items[key] = items[key].replace(dele, "")
    for rep in replace:
        if (rep in items[key]):
            items[key] = items[key].replace(rep, replace[rep])
    for it in add:
        if (it in items[key]):
            items[key] = items[key].replace(it, " "+it+" ")
    # se agrega un espacio en blanco antes de \\ para que pueda diferenciarse a los comandos del resto de letras o numeros
    if ('\\' in items[key]):
        items[key] = items[key].replace('\\', " \\")
    # se separa el ground truth, siendo la condicion para separa el que existan uno o mas espacios en blanco entre caracteres
    items[key] = re.split(r'\s+', items[key])
    count = 0
    # se itera sobre la lista generada de letras, numeros, simbolos y comandos para cada archivo inkml
    # separando los caracteres que se encuentren juntos y no sean parte de un comando, 
    # por ejemplo 'abc' se separa en 'a', 'b', 'c'
    for a in items[key]:
        if ('\\' not in a and len(a)>1):
            uno = items[key][:count]
            dos = re.split('', a)[1:-1]
            tres = items[key][count+1:]
            count+= len(dos)-1
            uno.extend(dos)
            uno.extend(tres)
            items[key] = uno
        count+=1
    if (items[key][-1]==""):
        items[key] = items[key][:-1]
    if (items[key][0]==""):
        items[key] = items[key][1:]
# se guarda en labels.json
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'w') as f:
    json.dump(items, f, indent=4)

In [2]:
# se define el numero maximo de palabras a tokenizar
num_words = 1000
# token para labels desconocidas
oov_token = '<UNK>'
# parametros de inicializacion del tokenizer de tensorflow
pad_type = 'post'
trunc_type = 'post'

items = {}
# se obtienen las listas de labels (numeros, comandos latex, simbolos y letras) para cada archivo inkml
# derivados del preprocesamiento previo
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'r') as f:
    items = json.load(f)
tokens = {}
count = 0
# se itera para cada archivo
for key in items:
    # se convierte a string
    items[key] = ' '.join(map(str, items[key]))
    # se anade al inicio y final los labels start y end para indicar inicio y final del ground truth
    items[key] = '<start> '+items[key] + ' <end>'
    # se vuelven a separar
    items[key] = re.sub(r"\s+", " ", items[key])
# se obtiene la cantidad de archvios inkml a tratar
keys = list(items.keys())
# se anade el groud truth de cada archivo a una lista
data = [ items[key] for key in keys ]
print(data[0])
# se instancia la funcion tokenizer con los parametros establecidos en un principio
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token, filters='', lower=False)
# se obtienen los tokens para el conjunto de datos, es decir, a cada comando de latex, letra, numero o simbolo
# utilizado se le asigna un numero entero en base a su frecuencia de aparicion
tokenizer.fit_on_texts(data)
# se define la cantidad de palabras o tokens
word_index = tokenizer.word_index
# se guardan los tokens en tokens.json
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\tokens.json", 'r') as f:
        word_index = json.load(f)
tokenizer.word_index = word_index
# se convierten los labels a tokens para cada archivo inkml
# por ejemplo ['a','b','c'] se convierte a [1,2,3] asumiendo que estos son sus tokens
train_sequences = tokenizer.texts_to_sequences(data)
# se obtiene la longitud del label mas grande
maxlen = max([len(x) for x in train_sequences])
# se asocian los tokens a cada archivo haciendo un pad hacia la maxima longitud
# tal que todos los archivos tenga por label una lista de la misma longitud, rellenando con ceros
# aquellos tokens que se encuentran en una longitud mayor al verdadero para determinado archivo
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

<start> S = ( \sum _ { i = 1 } ^ { n } \theta _ i - ( n - 2 ) \pi ) r ^ 2 <end>


In [ ]:
# se cargan los datos generados anteriormente: los tokens y etiquetas sin toquenizar de los
# archivos del dataset
LABELS_PATH = r"/content/drive/My Drive/Colab Notebooks/TrainINKML"
tokens = {}
files_imgs = {}
with open("/content/drive/My Drive/Colab Notebooks/TrainINKML/tokens.json", 'r') as f:
    tokens = json.load(f)
tokensPerFile = {}
items = {}
with open("/content/drive/My Drive/Colab Notebooks/TrainINKML/labels.json", 'r') as f:
    items = json.load(f)
# se guardan los tokens por archivo inkml haciendo referencia ahora a la imagen 
# generada a partir del archvio inkml
for key in items:
    newKey = LABELS_PATH +'/images/'+ key[len(LABELS_PATH)+15:][:-6].replace('\\','/')+'.jpg'
    tokensPerFile[newKey] = []
    files_imgs[newKey] = LABELS_PATH + key[43:].replace('\\','/')
    for command in items[key]:
        tokensPerFile[newKey].append(tokens[command])
# se guarda el conjunto de datos tokenizado en un formato json para su posterior uso
with open("/content/drive/My Drive/Colab Notebooks/TrainINKML/labelsPerFile.json", 'w') as f:
    json.dump(tokensPerFile, f, indent=4)
with open("/content/drive/My Drive/Colab Notebooks/TrainINKML/files_img.json", 'w') as f:
    json.dump(files_imgs, f, indent=4)

# Estructura del modelo

In [3]:
# se define la arquitectura del encoder en base a Zhang (2017)
class FCN_encoder(tf.keras.Model):
    def __init__(self, dropout_rate = 0.2):
        super(FCN_encoder, self).__init__()
        # super dentro del constructor permite que la clase herede y se convierta en un objeto de Keras
        
        # bloque de convolucion 1, 32 filtros
        # cada capa convolucional es seguida por un batch normalization y una capa de activacion
        # relu, el movimiento del kernel de convolucion se establece como 1 y
        # la dimension del kernel se establece como 3x3
        self.conv_1_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.batch_1_1 = tf.keras.layers.BatchNormalization()
        self.act_1_1 = tf.keras.layers.Activation('relu')
        self.conv_1_2 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.batch_1_2 = tf.keras.layers.BatchNormalization()
        self.act_1_2 = tf.keras.layers.Activation('relu')
        self.conv_1_3 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.batch_1_3 = tf.keras.layers.BatchNormalization()
        self.act_1_3 = tf.keras.layers.Activation('relu')
        self.conv_1_4 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.batch_1_4 = tf.keras.layers.BatchNormalization()
        self.act_1_4 = tf.keras.layers.Activation('relu')
        
        # maxpooling para reducir el tamaño
        self.maxPool_1 = tf.keras.layers.MaxPooling2D()
        
        # bloque convolucional 2, 64 filtros
        self.conv_2_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_2_1 = tf.keras.layers.BatchNormalization()
        self.act_2_1 = tf.keras.layers.Activation('relu')
        self.conv_2_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_2_2 = tf.keras.layers.BatchNormalization()
        self.act_2_2 = tf.keras.layers.Activation('relu')
        self.conv_2_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_2_3 = tf.keras.layers.BatchNormalization()
        self.act_2_3 = tf.keras.layers.Activation('relu')
        self.conv_2_4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_2_4 = tf.keras.layers.BatchNormalization()
        self.act_2_4 = tf.keras.layers.Activation('relu')

        self.maxPool_2 = tf.keras.layers.MaxPooling2D()

        self.conv_3_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_3_1 = tf.keras.layers.BatchNormalization()
        self.act_3_1 = tf.keras.layers.Activation('relu')
        self.conv_3_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_3_2 = tf.keras.layers.BatchNormalization()
        self.act_3_2 = tf.keras.layers.Activation('relu')
        self.conv_3_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_3_3 = tf.keras.layers.BatchNormalization()
        self.act_3_3 = tf.keras.layers.Activation('relu')
        self.conv_3_4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_3_4 = tf.keras.layers.BatchNormalization()
        self.act_3_4 = tf.keras.layers.Activation('relu')

        self.maxPool_3 = tf.keras.layers.MaxPooling2D()

        self.conv_4_1 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1)
        self.batch_4_1 = tf.keras.layers.BatchNormalization()
        self.act_4_1 = tf.keras.layers.Activation('relu')
        self.conv_4_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1)
        self.batch_4_2 = tf.keras.layers.BatchNormalization()
        self.act_4_2 = tf.keras.layers.Activation('relu')
        self.conv_4_3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1)
        self.batch_4_3 = tf.keras.layers.BatchNormalization()
        self.act_4_3 = tf.keras.layers.Activation('relu')
        self.conv_4_4 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1)
        self.drop_4 = tf.keras.layers.Dropout(dropout_rate)
        self.batch_4_4 = tf.keras.layers.BatchNormalization()
        self.act_4_4 = tf.keras.layers.Activation('relu')

        self.maxPool_4 = tf.keras.layers.MaxPooling2D()
    # funcion de llamada de la clase, dentro de la cual se establece la secuencia de layers que el inpput seguirá
    # cada 'bloque' de codigo representa al conjunto de layers que comparten la misma cantidad de filtros en el
    # kernel de las capas convolucionales
    def call(self, inputs):
        
        x = self.conv_1_1(inputs)
        x = self.batch_1_1(x)
        x = self.act_1_1(x)
        x = self.conv_1_2(x)
        x = self.batch_1_2(x)
        x = self.act_1_2(x)
        x = self.conv_1_3(x)
        x = self.batch_1_3(x)
        x = self.act_1_3(x)
        x = self.conv_1_4(x)
        x = self.batch_1_4(x)
        x = self.act_1_4(x)
        x = self.maxPool_1(x)
        
        x = self.conv_2_1(x)
        x = self.batch_2_1(x)
        x = self.act_2_1(x)
        x = self.conv_2_2(x)
        x = self.batch_2_2(x)
        x = self.act_2_2(x)
        x = self.conv_2_3(x)
        x = self.batch_2_3(x)
        x = self.act_2_3(x)
        x = self.conv_2_4(x)
        x = self.batch_2_4(x)
        x = self.act_2_4(x)
        x = self.maxPool_2(x)
        
        x = self.conv_3_1(x)
        x = self.batch_3_1(x)
        x = self.act_3_1(x)
        x = self.conv_3_2(x)
        x = self.batch_3_2(x)
        x = self.act_3_2(x)
        x = self.conv_3_3(x)
        x = self.batch_3_3(x)
        x = self.act_3_3(x)
        x = self.conv_3_4(x)
        x = self.batch_3_4(x)
        x = self.act_3_4(x)
        x = self.maxPool_3(x)
        
        x = self.conv_4_1(x)
        x = self.batch_4_1(x)
        x = self.act_4_1(x)
        x = self.conv_4_2(x)
        x = self.batch_4_2(x)
        x = self.act_4_2(x)
        x = self.conv_4_3(x)
        x = self.batch_4_3(x)
        x = self.act_4_3(x)
        x = self.conv_4_4(x)
        x = self.drop_4(x)
        x = self.batch_4_4(x)
        x = self.act_4_4(x)
        x = self.maxPool_4(x)
        # el encoder retorna el feature map generado par la ultima capa de max pooling
        return x
    # se instancia una clase que permite conocer el tamaño de los parametros dentro de la red.
    def model(self):
        input = tf.keras.layers.Input(shape=(150, 180, 1))
        return tf.keras.Model(inputs = input, outputs = self.call(input) )
print(FCN_encoder().model().summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 180, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 178, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 148, 178, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 148, 178, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 176, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 146, 176, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 146, 176, 32)      0     

In [4]:
# el modelo de atencion es instanciado
class Attender(tf.keras.Model):
    def __init__(self):
        super(Attender, self).__init__()
        # se instancia los distintos dense layers parametrizados en Zhang
        
        # representa la capa que se utiliza para aprender del hidden state anterior del decoder
        self.W_1 = tf.keras.layers.Dense(128)
        # capa que aprendera del feature map generado por el encoder
        self.U_a = tf.keras.layers.Dense(128)
        # la dimension de atencion es 128
        self.V_a = tf.keras.layers.Dense(128)
    
    def call(self, a, h):
        # se espanden dimensiones para el hidden state
        h_t = tf.expand_dims(h, 1)
        # se calcula el estado intermedio llamando a los dense layers y la funcion de tangente 
        # hiperbolica con la suma de los dos dense layers que reciben como parametro el hidden state
        # y el vector de anotacion
        e_ti = self.V_a( (tf.nn.tanh( self.W_1(h_t) + self.U_a(a))))
        # se aplica la activacion softmax al resultado
        a_ti = tf.nn.softmax(e_ti)
        # se calcula el vector de contexto multiplicando los coeficientes a_ti por el vector de anotacion
        context = a_ti * a
        # se obtiene la suma del resutado de la multiplicacion anterior
        context = (tf.reduce_sum(tf.reduce_sum(context, axis =1), axis=1))
        # se regresa el vector de contexto y los coeficientes a_ti
        return context     

In [5]:
# se define el modelo decoder que utiliza el gru
class GRU_decoder(tf.keras.Model):
    def __init__(self, dimension, units, label_len):
        super(GRU_decoder, self).__init__()
        # las unidades del gru
        self.units = units
        # la capa de embedding, establecida en Zhang como E
        self.embedding = tf.keras.layers.Embedding(label_len, dimension)
        # la capa del gru, se especifica que se desea el regreso de las secuencias y el ultimo estado calculado
        self.gru = tf.keras.layers.GRU( self.units, return_sequences=True, return_state = True, recurrent_initializer='glorot_uniform')
        # dense layers para el calculo de las probabilidades para cada palabra en el diccionario de palabras
        # es decir, los tokens
        # dense layer utilizado para la salida del GRU
        self.fc1 = tf.keras.layers.Dense(128)
        # dense layer utilizado para aprender del vector de contexto generado
        self.fc2 = tf.keras.layers.Dense(128)
        # dense layer utilizado para aprender de la salida anterior del decoder
        self.fc3 = tf.keras.layers.Dense(128)
        # la ultima capa se define con una activacion softmax
        self.fc4 = tf.keras.layers.Dense(128, activation='softmax')
        # se instancia el modelo de atencion
        self.attention = Attender()
      
    def call(self, a,x, h):
        # la llamada recibe la salida del encoder, la entrada del decoder (su salida anterios) 
        # y el estado anterior del decoder
        # se llama al modelo de atencion y se recibe el vector de contexto y los coeficientes de atencion
        context_v = self.attention(a,h)
        x = self.embedding(x)
        # se anade el contexto a la entrada del decoder para utilizar el conocimeinto previo
        t = tf.concat([context_v,x], axis =-1)
        # se expanden las dimensiones para hacer compatible el tensor con la entrada del GRU
        t = tf.expand_dims(t, 1)
        # se alimenta al gru con dicha informacion y se obtiene una salida y su estado actual
        out, state = self.gru(t)
        # se reduce la dimension de la salida del gru para hacer posible su 
        out = tf.reduce_sum(out, axis=1)
        
        # es necesario calcular las probabilidades condicionales, por lo que se utiliza 
        # la metodologia descrita por zhang al sumar las salidas de las capas 
        # que usan la informacion de distintias secciones del modelo encoder-decoder
        # como entrada de una capa final en la cual se realiza la clasficacion
        a = self.fc1(out)
        b = self.fc2(context_v)
        c = self.fc3(x)
        # se suman los resultados de cada capa densa
        res = a+b+c
        # se obtienen las probabilidades gracias al softmax del ultimo layer
        res = self.fc4(res)
        # se regresan las probabilidades, el estado y los coeficientes de atencion
        # las probabilidades corresponden a un tensor que posee las siguientes dimensiones:
        # (tamano_batch, cantidad de palabras) donde la cantidad de palabras es la misma
        # que los tokens producidos durante el preprocesamiento.
        return res, state

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    # funcion especial para inicializar la entrada del GRU por primera vez como un tensor
    # de ceros, establecido como la forma mas eficiente de inicializar este t
    def reset(self, batch):
        return tf.zeros((batch,3, self.units))

# Definicion de funciones para entrenamiento

In [6]:
# instanciacion del encoder
encoder = FCN_encoder()
# isntanciacion del decoder, con valores para la dimension del embedding, del gru y la cantidad de tokens
decoder = GRU_decoder(128,128,len(list(word_index.keys()))+1)
# se define una funcion de optimizacion para el modelo, el cual permite realizar la optimizacion en base
# al gradiente de los errores
optimizer = tf.keras.optimizers.Adam()
# definimos el tipo de calculo para los errores o perdiada entre los correspondientes labels del ground 
#truth de cada imagen y las predicciones realizadas por el modelo
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

In [7]:
# se define una fucnion de loss (perdida o error) que recibe como parametro un label del ground truth por imagen 
# en el batch las matrices de prediccion asociadas a cada una de esas imagenes
def loss_function(real, pred):
    # se realiza una conversion del tipo de dato para las probabilidades de las predicciones
    pred = tf.cast(pred, tf.float32)
    # se establece una mascara para aquellas imagenes cuyo label sea 0, es decir, que
    # es el padding agregado y que de esta forma no se considere su error para el error general
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    # se calcula el error
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    # se anade la mascara al los obtenido
    loss_ *= mask
    # se retorna la media de las perdidas para las imagenes en el batch
    return tf.reduce_mean(loss_)

In [8]:
# se define una funcion para el calculo de presicion debido a que el padding en el groundTruth de las
# imagenes implica que los ceros utilizados para ello seran tomados en cuenta durante su calculo
# con la funcion propia de tensorflow
def accuracy(groundTruth, pred):
    # recibe los labels correctos del batch y las predicciones para cada una de las clases (127)
    # se obtiene el argmax o el mayor valor de cada prediccion
    pred_values = K.cast(K.argmax(pred, 1), dtype='int32')
    # se establece cuales de dichas predicciones son verdaderas con base al groundTruth o etiqueta
    correct = K.cast(K.equal(groundTruth, pred_values), dtype='float32')
    # se utiliza una mascara para establecer aquellos espacios dentro de la etiqueta que representan un
    # cero utilizado para el padding
    mask = K.cast(K.greater(groundTruth, 0), dtype='float32')
    # se toman en cuenta solamente aquellos valores que no sean cero
    n_correct = K.sum(mask * correct)
    # se calcula la cantidad de etiquetas del groundTruth que no son padding
    n_total = K.sum(mask)
      # se retorna el ratio representando la fraccion de predicciones correctas para esta seccion del batch
    return n_correct / n_total

In [9]:
data = {}
# se obtienen los tokens por imagen
LABELS_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labelsPerFile.json"
with open(LABELS_PATH,  'r') as f:
    data = json.load(f)
    f.close()

In [10]:
files_img = {}
# se obtienen los nombres de las imagenes
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\files_img.json", 'r') as f:
    files_img = json.load(f)
    f.close()
keys = list(files_img.keys())
files = list(data.keys())
# se obtiene el dataset a partir del diccionario que contiene los tokens por archivo
list_ds = tf.data.Dataset.list_files(files)

# funcion que obtiene la imagen y su label, tiene como parametro el path de la imagen
def getImages(f):
    # se obtiene el indice del archivo para obtener sus labels (tokens)
    index = keys.index(f)
    # se lee la imagen desde su fuente
    image = tf.io.read_file(f)
    # se decodifica el formato jpg y se establece como imagen en un solo canal, es decir
    # blanco y negro
    image = tf.image.decode_jpeg(image, channels = 1)  
    # se convierte su tipo de dato a flotante
    image = tf.image.convert_image_dtype(image, tf.float32)
    # se redimensiona para que concuerde con el input del modelo (encoder)
    image = tf.image.resize(image, [150, 180] ) 
    # se retorna la imagen (su matriz de pixels) y su label
    return image, train_padded[index]

# instancia el dataset a utilizar como un mapeo de los archvios dentro de list_ds que manda llamar a la funcion getProcessedImages
labeledDataset = list_ds.map(lambda x: tf.py_function(getImages, [x], [tf.float32, tf.int32]), num_parallel_calls=tf.data.AUTOTUNE)

# solo para verificar que funciona, imprime las dimensioens de una imagen del dataset y su label
for image, label in labeledDataset.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (150, 180, 1)
Label:  [ 4 23  2 10 22 27  3 15 23  2 10  3 23  2 22  3 23  2 27  3  5  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [11]:
# se utiliza un shuffle para revolver las imagenes dentro del dataset y evitar
# que imagenes continuas siempre sean tomadas dentro del mismo batch
labeledDataset = labeledDataset.shuffle(buffer_size=20)
# se establece el tamano de batch como 8, es decir, se ingresara al entrenamiento paquetes de 8 imagenes
labeledDataset = labeledDataset.batch(8)

# Entrenamiento

In [12]:
import random
# se define la funcion que entrena al modelo de encoder-decoder
# recibe como parametros el batch de imagenes y su ground truth (labels)
def train(image, groundTruth):
    loss = 0
    acc = []
    # el estado del gru se inicializa como ceros
    hidden = decoder.reset(groundTruth.shape[0])
    # el input del decoder se inicializa como un tensor con valores para el primer token en todas las imagenes
    input_decoder = tf.constant([word_index['<start>']] * groundTruth.shape[0])
    g1 = tf.random.Generator.from_seed(1)
    #input_decoder = tf.expand_dims([[word_index['<start>']]*3] * groundTruth.shape[0], 1)
    #print(input_decoder.shape, hidden.shape)
    # ciclo que permite el entrenamiento al generar un entorno donde las variables de entrenamiento son 'vigiladas'
    # durante el entrenamiento para poder corregirlas y ajusstar el modelo
    count = 0
    with tf.GradientTape() as tape:
        # se llama al encoder con la imagen la cual regresa el feature map con las caracteristicas extraidas de la imagen
        feature_map = encoder(image)
        # se itera por la cantidad de posibles palabras en un groundTruth
        for i in range(1, groundTruth.shape[1]):
            # para cada palabra se verifica si su tensor corresponde a solo valores en cero, para lo cual se rompe el ciclo
            # terminando el entrenamiento de este batch
            sum_ = tf.reduce_sum(groundTruth[:,i])
            allZero = tf.equal(sum_, 0)
            if (allZero):
                break
            # si existe por lo menos un label por predecir entonces se llama al decoder envindole 
            # su respectivo input, las salidas del encoder y el hidden state anterior
            pred, hidden = decoder(feature_map,input_decoder, hidden)
            # se calcula el error de las predicciones y se suma al actual para el batch
            loss += loss_function(groundTruth[:, i], pred)
            acc.append(accuracy(groundTruth[:, i], pred).numpy())
            #print(i,"loss:", loss)
            # la siguiente entrada para el decoder son las anotaciones para la iteracion actual
            # es decir, la anterior para la siguiente interacion (h-1)
            input_decoder = groundTruth[:, i]
            count +=1
    # se obtienen las variables a las que se les puede modificar los parametros para ajustarlas (entrenarlas)
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    # se obtiene el gradiente de error en base al error obtenido
    gradients = tape.gradient(loss, trainable_variables)
    # se realiza el ajuste en base al gradiente de error
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    # una vez que termina la entrada del batch se calcula el error total con el error obtenido
    total_loss = (loss / count)
    # se regresa el error dividido entre el numero de iteraciones para el batch,
    # el error total del batch (loss), asi como la media de la presicion en las predicciones del batch
    return loss, total_loss, sum(acc)/len(acc)

In [13]:
# funcion que itera sobre el dataset para entrenarlo
def trainData(dataset, loadEpoch, batchToLoad, load, EPOCHS = 2):
    print(len(dataset))
    # se itera por las epocas de entrenamiento
    for epoch in range(EPOCHS):
        # se inicializa la perdida como 0 para cada epoca de entrenamiento
        total_loss = 0
        num = 0
        # se hace iterable el dataset para obtener cada batch de images de forma individual
        # obteniendo el numero de batch, el tensor de imagenes (pixeles) y las etiquetas de cada imagen del batch
        for (batch, (img_tensor, target)) in enumerate(dataset):
            # se llama a la funcion de entrenamiento para cada batch enviando el tensor de imagenes y de labels
            # obteniendo los erroes
            print("batch",batch+1, end=" ")
            # dado que el ultimo batch tiene longitud distinta, se omite en cada epoca
            if (batch+1 == 1023):
                break
            # se llama a la funcion de entrenamiento, enviando como parametro el batch de imagenes y labels
            batch_loss, t_loss, acc = train(img_tensor, target)
            
            if (load):
                break
            # utilizado para guardar los pesos del encoder y decoder cada 100 batches
            if ((batch+1)%100 == 0 ):
                encoder.save_weights(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\train4\enc\{0}_{1}.h5".format(loadEpoch+1+epoch, batch),
                    save_format='h5')
                decoder.save_weights(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\train4\dec\{0}_{1}.h5".format(loadEpoch+1+epoch, batch),
                    save_format='h5')
            print("loss:", t_loss,"accuracy:",acc , end="\n")
            # se suma la perdida obtenida en cada batch
            total_loss += t_loss
            num +=1
        if not load:    
            print('Epoch {0:d}/{1:d}'.format(epoch+1, EPOCHS))
            print('===============>  train-loss=%.3f' % (total_loss/num))
        else:
            break
        
       

In [15]:
# se llama la funcion principal de entrenamiento para inicializar
trainData(labeledDataset, 7, 999, True, 5)
# se cargan los pesos de un entrenamiento anterior
encoder.load_weights(r"F:\MEGA\CETYS\fünf\Machine Learning\enc.h5")
decoder.load_weights(r"F:\MEGA\CETYS\fünf\Machine Learning\dec.h5")
# se llama a la funcion de entrenamiento para entrenar haciendo uso de los pesos
# obtenidos de los entrenamientos previos, continuando el aprendizaje
trainData(labeledDataset, 3, 999, False, 2)

1023
batch 1 1023
batch 1 loss: tf.Tensor(4.3818984, shape=(), dtype=float32) accuracy: 0.5008365556194976
batch 2 loss: tf.Tensor(4.431326, shape=(), dtype=float32) accuracy: 0.5304357651269661
batch 3 loss: tf.Tensor(4.109344, shape=(), dtype=float32) accuracy: 0.46498599823783426
batch 4 loss: tf.Tensor(3.4599683, shape=(), dtype=float32) accuracy: 0.5295995704152368
batch 5 loss: tf.Tensor(2.5593164, shape=(), dtype=float32) accuracy: 0.38543651103973386
batch 6 

KeyboardInterrupt: 

# Testing

In [16]:
# se define el numero maximo de palabras a tokenizar
num_words = 1000
# token para labels desconocidas
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

items = {}
# se obtienen las listas de labels (numeros, comandos latex, simbolos y letras) para cada archivo inkml

with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TestINKML\labels.json", 'r') as f:
    items = json.load(f)
for key in items:
    # se convierte a string
    items[key] = ' '.join(map(str, items[key]))
    # se anade al inicio y final los labels start y end para indicar inicio y final del ground truth
    items[key] = '<start> '+items[key] + ' <end>'
    # se vuelven a separar
    items[key] = re.sub(r"\s+", " ", items[key])
# se obtiene la cantidad de archvios inkml a tratar
keys = list(items.keys())
# se anade el groud truth de cada archivo a una lista
data = [ items[key] for key in keys ]
# se instancia la funcion tokenizer con los parametros establecidos en un principio
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token, filters='', lower=False)

with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\tokens.json", 'r') as f:
        word_index = json.load(f)
tokenizer.word_index = word_index
# se convierten los labels a tokens para cada archivo inkml
# por ejemplo ['a','b','c'] se convierte a [1,2,3] asumiendo que estos son sus tokens
test_sequences = tokenizer.texts_to_sequences(data)
# se obtiene la longitud del label mas grande
maxlen = max([len(x) for x in train_sequences])
# se asocian los tokens a cada archivo haciendo un pad hacia la maxima longitud
# tal que todos los archivos tenga por label una lista de la misma longitud, rellenando con ceros
# aquellos tokens que se encuentran en una longitud mayor al verdadero para determinado archivo
test_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

In [17]:
data= {}
LABELS_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TestINKML\labelsPerFile.json"
with open(LABELS_PATH,  'r') as f:
    data = json.load(f)
    f.close()
files_img = {}
# se obtienen los nombres de las imagenes
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TestINKML\files_img.json", 'r') as f:
    files_img = json.load(f)
    f.close()
keys = list(files_img.keys())
files = list(data.keys())
# se obtiene el dataset a partir del diccionario que contiene los tokens por archivo
list_ds = tf.data.Dataset.list_files(files)

# funcion que obtiene la imagen y su label, tiene como parametro el path de la imagen
def getImagesTest(f):
    index = keys.index(f)
    # se lee la imagen
    image = tf.io.read_file(f)
    # se decodifica el formato jpg y se establece como imagen en un solo canal, es decir
    # blanco y negro
    image = tf.image.decode_jpeg(image, channels = 1)  
    # se convierte su tipo de dato a flotante
    image = tf.image.convert_image_dtype(image, tf.float32)
    # se redimensiona para que concuerde con el modelo del encoder
    image = tf.image.resize(image, [150, 180] ) 
    # se retorna la imagen (su matriz de pixels) y su label
    return image, test_padded[index]

# define el dataset a utilizar como un mapeo de los archvios dentro de list_ds que manda llamar a la funcion getImagesTest
testDataset = list_ds.map(lambda x: tf.py_function(getImagesTest, [x], [tf.float32, tf.int32]), num_parallel_calls=tf.data.AUTOTUNE)

# solo para verificar que funciona, imprime las dimensioens de la imagen y su label
for image, label in testDataset.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (150, 180, 1)
Label:  [ 4 85 15 13 39 11  2 29 15  6  3  8  2 19  3 46 11 29 12 13 19 12  7 14
 47 14 31  8  7  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [18]:
# se hace shuffle al conjunto de daots de prueba y se establece un tamano de batch 
# similar al de entrenamiento
testDataset = testDataset.shuffle(buffer_size=20)
testDataset = testDataset.batch(8)
print(len(testDataset))

84


In [27]:
test_data = []
def test(image, groundTruth):
    global glob_hidden
    loss = 0
    acc = []
    temp = []
    # el estado del gru se inicializa como ceros
    hidden = decoder.reset(groundTruth.shape[0])
    # el input del decoder se inicializa como un tensor con valores para el primer token en todas las imagenes
    input_decoder = tf.constant([word_index['<start>']] * groundTruth.shape[0])
    count = 0
    # se llama al encoder con la imagen la cual regresa el feature map con las caracteristicas extraidas de la imagen
    feature_map = encoder(image)
    # se itera por la cantidad de posibles palabras en un groundTruth
    for i in range(1, groundTruth.shape[1]):
        # para cada palabra se verifica si su tensor corresponde a solo valores en cero, para lo cual se rompe el ciclo
        # terminando la prueba de este batch
        sum_ = tf.reduce_sum(groundTruth[:,i])
        allZero = tf.equal(sum_, 0)
        if (allZero):
            break
        # si existe por lo menos un label por predecir entonces se llama al decoder envindole 
        # su respectivo input, las salidas del encoder y el hidden state anterior
        pred, hidden = decoder(feature_map, input_decoder, hidden)
        temp.append([groundTruth[:,i], K.argmax(pred, 1)])
        # se calcula el error de las predicciones y se suma al actual para el batch
        loss += loss_function(groundTruth[:, i], pred)
        acc.append(accuracy(groundTruth[:, i], pred).numpy())
        #print(i,"loss:", loss)
        # la siguiente entrada para el decoder son las anotaciones para la iteracion actual
        # es decir, la anterior para la siguiente interacion (h-1)
        input_decoder = groundTruth[:, i]
        count +=1
    test_data.append(temp)
    # dado que es la prueba, se elimina la necesidad de calcular gradientes y ajustar los pesos
    # de los distintos layers del modelo
    # una vez que termina la entrada del batch se calcula el error total con el error obtenido
    total_loss = (loss / count)
    # se regresa el error, el error total y la presicion
    return loss, total_loss, sum(acc)/len(acc)

In [28]:
def testData(dataset):
    print(len(dataset))
    # se itera por las epocas de entrenamiento
    total_loss = 0
    num = 0
    # para cada batch de images en el dataset
    for (batch, (img_tensor, target)) in enumerate(dataset):
        # se llama a la funcion de entrenamiento para cada batch enviando el tensor de imagenes y de labels
        # obteniendo los erroes
        print("batch",batch+1, end=" ")
        if (batch+1 == 1023):
            break
        batch_loss, t_loss, acc = test(img_tensor, target)
        print("loss:", t_loss,"accuracy:",acc , end="\n")
        total_loss += t_loss
        num +=1
    print('===============>  train-loss=%.3f' % (total_loss/num))
        

In [29]:
testData(testDataset)

84
batch 1 loss: tf.Tensor(3.163328, shape=(), dtype=float32) accuracy: 0.6747023910284042
batch 2 loss: tf.Tensor(3.2258883, shape=(), dtype=float32) accuracy: 0.5390350948038855
batch 3 loss: tf.Tensor(2.468643, shape=(), dtype=float32) accuracy: 0.5084686184471304
batch 4 loss: tf.Tensor(2.2772663, shape=(), dtype=float32) accuracy: 0.6149891804565083
batch 5 loss: tf.Tensor(2.4477868, shape=(), dtype=float32) accuracy: 0.5775974077934568
batch 6 loss: tf.Tensor(2.9205372, shape=(), dtype=float32) accuracy: 0.6448529476628584
batch 7 loss: tf.Tensor(2.690727, shape=(), dtype=float32) accuracy: 0.6041805148124695
batch 8 loss: tf.Tensor(2.885446, shape=(), dtype=float32) accuracy: 0.6088359819518195
batch 9 loss: tf.Tensor(2.4384165, shape=(), dtype=float32) accuracy: 0.49202658236026764
batch 10 loss: tf.Tensor(2.67287, shape=(), dtype=float32) accuracy: 0.6006084693802728
batch 11 loss: tf.Tensor(2.9448988, shape=(), dtype=float32) accuracy: 0.5000000046359168
batch 12 loss: tf.Ten

In [146]:
print(glob_hidden.shape)

(7, 128)


In [30]:

final_labels = [[] for _ in range(671)]
final_pred = [[] for _ in range(671)]
index = 0
for i in test_data:
    for j in i:
        for k, count in zip(j[0].numpy(), range(8)):
            final_labels[index+count].append(k)
        for k, count in zip(j[1].numpy(), range(8)):
            final_pred[index+count].append(k)
    index+=8
        #final_labels[index].append(j[0].numpy())
WER_ = 0
for k in range(671):
    a = ' '.join([str(i) for i in final_labels[k]])
    b = ' '.join([str(i) for i in final_pred[k]])
    WER_ += wer(a,b)
print(WER_/671)

0.6872888732218218


In [214]:
k = np.random.randint(671)
print(final_labels[k])
original = parse_latex(' '.join([str(keys[i-1] ) for i in final_labels[k][:final_labels[k].index(5)]]))
original


[31, 13, 31, 11, 21, 32, 13, 46, 12, 46, 11, 21, 14, 12, 31, 11, 6, 32, 13, 46, 12, 46, 11, 6, 14, 14, 15, 31, 11, 21, 31, 11, 6, 32, 13, 46, 11, 6, 12, 46, 11, 21, 14, 5]
ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2


In [215]:
print(' '.join([str(keys[i-1] ) for i in final_pred[k][:final_pred[k].index(5)]]))
try:
    prediction_lat = parse_latex(' '.join([str(keys[i-1] ) for i in final_pred[k][final_pred[k].index(5)]]))
    prediction_lat
except:
    print('error')

\frac = x \cos { \sin C x } \theta _ 0 )
error


In [47]:
import math
from sympy import *
from sympy.parsing.latex import parse_latex
import antlr4
def evaluation(image):
    hidden = decoder.reset(groundTruth.shape[0])


image = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TestINKML\images\103_em_3.jpg"
img_val = tf.expand_dims(getProcessedImages2(image),0)
g2 = tf.random.get_global_generator()

image2 = r"F:\OneDrive\Imágenes\22.png"
img_val2 = tf.expand_dims(getProcessedImages2(image2),0)
input_decoder = tf.constant([4] * 1)
preds = []
hidden = tf.constant([[[0.0]*128]*3]*1)


batch = img_val
feature_map = encoder.predict(batch)
#batch = tf.concat([img_val, img_val2], 1)
'''pred, hidden = decoder(feature_map, input_decoder, hidden)
hypotheses = tf.math.top_k(pred, 10)
input_decoder = tf.argmax(pred, 1)
print(hypotheses.indices.numpy())
a = np.random.randint(127)
b = np.random.randint(127)
pred, hidden = decoder(feature_map,tf.constant([13, 4]) , hidden)
hypotheses = tf.math.top_k(pred, 10)
print(hypotheses.indices.numpy())
'''
init_printing()
for i in range(90):
    pred, hidden = decoder(feature_map, input_decoder, hidden)
    input_decoder = K.argmax(pred, 1)
    preds.append(input_decoder)
    #print(input_decoder.numpy()[0], end=" " )
    if (input_decoder.numpy()[0] == 5):
        break
keys = list(word_index.keys())
render = ''
for i in preds:
    print(i.numpy()[0], end=" ")
    render += keys[i.numpy()[0]-1] 
render = parse_latex(str(render))
render
#[16  2 13 23 10 41 17 64  6 20]

16 2 6 3 2 23 2 7 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2


In [22]:
def getProcessedImages2(f):
    #index = keys.index(f)
    # se lee la imagen
    image = tf.io.read_file(f)
    # se decodifica el formato jpg y se establece como imagen en un solo canal, es decir
    # blanco y negro
    image = tf.image.decode_jpeg(image, channels = 1, dct_method = 'INTEGER_ACCURATE')  
    # se convierte su tipo de dato a flotante
    image = tf.image.convert_image_dtype(image, tf.float32)
    # se redimensiona para que concuerde con el modelo del encoder
    image = tf.image.resize(image, [150, 180] ) 
    # se retorna la imagen (su matriz de pixels) y su label
    return image#, train_padded[index] #[data[str(tf.constant(f).numpy())[2:-1].replace('\\\\', '\\') ]]

In [169]:
keys[96] = "\\lt"
keys[105] = "\\gt"
keys[14]='='
keys[97] = '\\!'
keys[39] = '\\,'